In [8]:
# import importlib.util
import importlib
import pandas as pd
import numpy as np
import torch
import sys
import os

%load_ext autoreload
%autoreload 2

sys.path.append('../code/src/')
import utils


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### CSV creation from stations

In [18]:
# Shamat
shamat_df = utils.get_df_from_shamat_csv('../data/raw/shamat/shamat_beit_dagan.csv')
radiation_df = utils.get_df_from_radiation_csv(('../data/raw/shamat/radiation_beit_dagan.csv'))
shamat_df['rad'] = radiation_df['rad']

if not os.path.exists('../data/processed_csvs/'):
    os.makedirs('../data/processed_csvs/')
shamat_df.to_csv('../data/processed_csvs/shamat.csv', index=True)

/Users/avnerran/MSc/aps_analysis/transformers/notebooks/../code/src/utils.py:27: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  ts_col = df[0][0:last_raw_i]


In [19]:
shamat_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17640 entries, 2023-05-01 00:00:00 to 2023-08-31 12:00:00
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   shamat_P                       17640 non-null  float64
 1   shamat_RH                      17640 non-null  float64
 2   shamat_T                       17640 non-null  float64
 3   shamat_T_ground                17640 non-null  float64
 4   shamat_T_max                   17640 non-null  float64
 5   shamat_T_min                   17638 non-null  float64
 6   shamat_precipitation           17640 non-null  float64
 7   shamat_wind_direction          17630 non-null  float64
 8   shamat_wind_direction_std_dev  17630 non-null  float64
 9   shamat_wind_gust_direction     17630 non-null  float64
 10  shamat_wind_gust_speed         17630 non-null  float64
 11  shamat_wind_speed              17630 non-null  float64
 12  shamat_wind

In [ ]:
stations_w_f0 = 0
station_wo_f0 = 0

for file in os.listdir('../data/raw/ep/'):
    if file.endswith('.xlsx'):
        station_df = utils.get_df_from_excel_station_file(f'../data/raw/ep/{file}')

        # adding the factor0 to the csv:
        try:
            station_df['factor0'] = utils.calculate_factor0(station_df, shamat_df)
            if station_df['factor0'].notna().any():
                stations_w_f0 += 1
            else:
                station_wo_f0 += 1
            station_df.to_csv(f'../data/processed_csvs/{file[:-5]}.csv', index=True)
            
        except:
            print(f'probably no measurement for the factor calculation in station: {file[:-5]}')
            station_df.to_csv(f'../data/processed_csvs/{file[:-5]}.csv', index=True)
            
            continue

print(f'total num of station: {stations_w_f0 + station_wo_f0}')
print(f'num of stations with factor0: {stations_w_f0}')
print(f'num of stations without factor0: {station_wo_f0}')

In [ ]:
'''
 the dates of the sessions are:

4-5 May 2023
8-9 May 2023
23-24 May 2023

25-26 August 2023
28-29 August 2023
 '''

#### Pulling timestamps from pop tensor

In [ ]:
# getting the time of the flights:
# the time in the list is the time at the center of the flight
flight_times = torch.load('../data/raw/drone/pom_timestamps.pt', map_location='cpu', weights_only=False)
flight_time_list = flight_times.to_list()

#### Saving the stations as tensors

In [ ]:
if not os.path.exists('../data/station_tensors/'):
    os.makedirs('../data/station_tensors/')


for station in os.listdir('../data/processed_csvs/'):
    if 'shamat' in station:
        continue
    if station.endswith('.csv'):
        t = utils.station_csv_to_tensor(f'../data/processed_csvs/{station}', flight_time_list, max_features=20, max_measurements=5)
        torch.save(t, f'../data/station_tensors/{station[:-4]}.pt')

In [ ]:
# the tensors shapes are [P, L, Fs, M]: [66, 5, 20, 5] when the missing features and measurements per feature are nan values.

In [ ]:
# checking tensors integrity
tmp_t = torch.load('../data/station_tensors/ashdod_kala.pt')
tmp_t[1,1,:,:]

# we can see that we have nans when full features are missing and when some measurements are missing. vi!

#### Creating the metadata json files for each stations

In [ ]:
for station in os.listdir('../data/processed_csvs/'):
    if station.endswith('.csv'):
        station = station[:-4]
        meta = utils.save_tensor_metadata(f"../data/processed_csvs/{station}.csv", station, tensor_shape=[[66, 5, 20, 5]], output_dir='../data/metadata/')

Now we have a tensor for each station: [P, L, Fs, M].  
Plus a metadata json file for each station.  

Plus, we have the tensor of the Ozone measurements: [P, h, Fd].